In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/My Drive/Colab Notebooks/Github/SindyAutoencoders_project

/content/drive/My Drive/Colab Notebooks/Github/SindyAutoencoders_project


In [3]:
import os
import datetime
import pandas as pd
import numpy as np
from robi.example_spring import get_spring_data
from src.sindy_utils import library_size
from src.training import train_network
import tensorflow as tf

# Generate data

In [4]:
training_data = get_spring_data(100)
validation_data = get_spring_data(10)

# Set up model and training parameters

In [5]:
params = {}

params['input_dim'] = training_data['x'].shape[-1]
params['latent_dim'] = 1
params['model_order'] = 2
params['poly_order'] = 2
params['include_sine'] = False
params['library_dim'] = library_size(2*params['latent_dim'], params['poly_order'], params['include_sine'], True)

# sequential thresholding parameters
params['sequential_thresholding'] = True
params['coefficient_threshold'] = 0.1
params['threshold_frequency'] = 500
params['coefficient_mask'] = np.ones((params['library_dim'], params['latent_dim']))
params['coefficient_initialization'] = 'constant'

# loss function weighting
params['loss_weight_decoder'] = 1.0
params['loss_weight_sindy_x'] = 5e-4
params['loss_weight_sindy_z'] = 5e-5
params['loss_weight_sindy_regularization'] = 1e-5

params['activation'] = 'sigmoid'
params['widths'] = [128,64,32]

# training parameters
params['epoch_size'] = training_data['x'].shape[0]
params['batch_size'] = 1000
params['learning_rate'] = 1e-4

params['data_path'] = os.getcwd() + '/'
params['print_progress'] = True
params['print_frequency'] = 100

# training time cutoffs
params['max_epochs'] = 5001
params['refinement_epochs'] = 1001

# Run training experiments

In [6]:
num_experiments = 1
df = pd.DataFrame()
for i in range(num_experiments):
    print('EXPERIMENT %d' % i)

    params['coefficient_mask'] = np.ones((params['library_dim'], params['latent_dim']))

    params['save_name'] = 'spring_' + datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S_%f")

    tf.compat.v1.reset_default_graph()

    results_dict = train_network(training_data, validation_data, params)
    df = df.append({**results_dict, **params}, ignore_index=True)

df.to_pickle('experiment_results_' + datetime.datetime.now().strftime("%Y%m%d%H%M") + '.pkl')

EXPERIMENT 0
TRAINING
Epoch 0
   training loss 0.002731034066528082, (0.0026911483, 0.52640957, 0.00725854, 0.99359125)
   validation loss 0.003602056298404932, (0.0032659862, 0.49151534, 0.6031169, 0.99359125)
decoder loss ratio: 0.400510, decoder SINDy loss  ratio: 1.000018
Epoch 100
   training loss 0.0005675706779584289, (0.0005608091, 0.00016071499, 0.0072559644, 0.3125574)
   validation loss 0.0018754403572529554, (0.0015707246, 0.0007403617, 0.6031063, 0.3125574)
decoder loss ratio: 0.192619, decoder SINDy loss  ratio: 1.000001
Epoch 200
   training loss 0.000559683539904654, (0.0005552089, 0.0028133136, 0.00725658, 0.070564434)
   validation loss 0.0018707149429246783, (0.0015681405, 0.0062444317, 0.6031134, 0.070564434)
decoder loss ratio: 0.192302, decoder SINDy loss  ratio: 1.000013
Epoch 300
   training loss 0.00014649836521130055, (0.00012844437, 0.2618605, 0.006403617, 0.17591466)
   validation loss 0.0008989585330709815, (0.0005821892, 0.58728987, 0.5712913, 0.17591466)


<ipython-input-6-69edefec05a3>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({**results_dict, **params}, ignore_index=True)
